In [4]:
import json
from pprint import pprint
import hashlib
import binascii


def splitStringFromBack(word, border):
    reverse_word = (word[::-1])
    split_reverse = [reverse_word[i:i+border] for i in range(0, len(reverse_word), border)]
    return list(reversed([i[::-1] for i in split_reverse]))

# TODO: here the inputs are converted to littleEndian, and in the circuits back to bigEndian ??
def littleEndian(string):
    splited = [str(string)[i:i + 2] for i in range(0, len(str(string)), 2)]
    splited.reverse()
    return "".join(splited)

def string_to_u32(val: str) -> [int]:
    byte_array = bytes.fromhex(val)
    u32_array = [str(int.from_bytes(byte_array[i:i+4], "big")) for i in range(0,len(byte_array), 4)]
    return u32_array # return " ".join(u32_array)

def createZokratesInputFromBlock(block):
    version = littleEndian(block['versionHex'])
    little_endian_previousHash = littleEndian(block['previousblockhash']) if block['height'] > 0 else 64 * '0'
    little_endian_merkleRoot = littleEndian(block['merkleroot'])
    little_endian_time = littleEndian(hex(block['time'])[2:])
    little_endian_difficultyBits = littleEndian(block['bits'])
    nonce = hex(block['nonce'])[2:]
    nonce = '0' * (8 - len(nonce)) + nonce #ensure nonce is 4 bytes long
    little_endian_nonce = littleEndian(nonce)

    print(f"version: {version}\nlittle_endian_previousHash: {string_to_u32(little_endian_previousHash)}")

    header = version + little_endian_previousHash + little_endian_merkleRoot + little_endian_time + little_endian_difficultyBits + little_endian_nonce
    return header

# source form: https://www.herongyang.com/Bitcoin/Block-Data-Block-Hash-Calculation-in-Python.html
def bitcoin_hash(header:str) -> str:
    header = binascii.unhexlify(header)
    result_1024 = hashlib.sha256(header).digest()
    print(f"1024 hash: {binascii.hexlify(result_1024)}")
    hash_result_256 = hashlib.sha256(result_1024).digest()
    hash_result =  binascii.hexlify(hash_result_256)
    return bytes.hex(binascii.unhexlify(hash_result)[::-1])

data_path = "./tests/test_data/test_proof/test_correct_proofs/batch_size_2_nr_0.json"

with open(data_path) as json_data:
    d = json.load(json_data)
    json_data.close()
    

result = d['http_responses'][2][0]['result']
block = createZokratesInputFromBlock(result)
print(block)
hash_one = bitcoin_hash(block)
print(f"hash_one: {hash_one}\nblock_hash: {result['hash']}")
assert(hash_one == result['hash'])
block_array = string_to_u32(block)
print(block_array)
print(len(block_array))

version: 01000000
little_endian_previousHash: ['1877117962', '3069293426', '3248923206', '2925786959', '2468250469', '3780774044', '1758861568', '0']
010000006fe28c0ab6f1b372c1a6a246ae63f74f931e8365e15a089c68d6190000000000982051fd1e4ba744bbbe680e1fee14677ba1a3c3540bf7b1cdb606e857233e0e61bc6649ffff001d01e36299


TypeError: fromhex() argument must be str, not bytes

In [2]:
epoch_head = block_array[16:]
previousblockhash = string_to_u32(littleEndian(result['previousblockhash']))

print(len(epoch_head), epoch_head)
print(len(previousblockhash), previousblockhash)

4 ['1461927438', '1639736905', '4294901789', '31679129']
8 ['1877117962', '3069293426', '3248923206', '2925786959', '2468250469', '3780774044', '1758861568', '0']


In [3]:
data_path = "./tests/test_data/test_proof/test_correct_proofs/batch_size_2_nr_1.json"

with open(data_path) as json_data:
    next_d = json.load(json_data)
    json_data.close()
    

next_result = next_d['http_responses'][2][0]['result']
next_block = createZokratesInputFromBlock(next_result)
print(f"hash_one: {hash_one}\nblock_hash: {next_result['previousblockhash']}")
assert(hash_one == next_result['previousblockhash'])
hash_two = bitcoin_hash(next_block)
print(f"hash_two: {hash_two}\nblock_hash: {next_result['hash']}")
assert(hash_two == next_result['hash'])
next_block_array = string_to_u32(next_block)

' '.join(epoch_head + previousblockhash + block_array + next_block_array)

version: 01000000
little_endian_previousHash: ['1214311192', '3206223392', '3816723600', '4236919413', '339832791', '1364831110', '1754176131', '0']
hash_one: 00000000839a8e6886ab5951d76f411475428afc90947ee320161bbf18eb6048
block_hash: 00000000839a8e6886ab5951d76f411475428afc90947ee320161bbf18eb6048
1024 hash: b'9cbba91850f7347bc2f08d325af67165f77751ad85f909c85ae10340a3165b69'
hash_two: 000000006a625f06636b8bb6ac7b960a8d03705d1ace08b1a19da3fdcc99ddbd
block_hash: 000000006a625f06636b8bb6ac7b960a8d03705d1ace08b1a19da3fdcc99ddbd


'1461927438 1639736905 4294901789 31679129 1877117962 3069293426 3248923206 2925786959 2468250469 3780774044 1758861568 0 16777216 1877117962 3069293426 3248923206 2925786959 2468250469 3780774044 1758861568 0 2552254973 508274500 3149817870 535696487 2074190787 1410070449 3451258600 1461927438 1639736905 4294901789 31679129 16777216 1214311192 3206223392 3816723600 4236919413 339832791 1364831110 1754176131 0 3590179924 505798172 2052775405 4064827576 3144047775 921662542 3828101472 583602075 2965136969 4294901789 148028769'

In [4]:
[int(x, 16) for x in ["0x404dffc7","0x78f925e0","0x5c4665d4","0xeb582a32","0x82abe580","0xbd0cd851","0x33d8ef15","0x2ba8450e"]]

[1078853575,
 2029594080,
 1548117460,
 3948423730,
 2192303488,
 3171735633,
 869854997,
 732448014]

In [5]:
result

{'bits': '1d00ffff',
 'chainwork': '0000000000000000000000000000000000000000000000000000000200020002',
 'confirmations': 652861,
 'difficulty': 1.0,
 'hash': '00000000839a8e6886ab5951d76f411475428afc90947ee320161bbf18eb6048',
 'height': 1,
 'mediantime': 1231469665,
 'merkleroot': '0e3e2357e806b6cdb1f70b54c3a3a17b6714ee1f0e68bebb44a74b1efd512098',
 'nTx': 1,
 'nextblockhash': '000000006a625f06636b8bb6ac7b960a8d03705d1ace08b1a19da3fdcc99ddbd',
 'nonce': 2573394689,
 'previousblockhash': '000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f',
 'size': 215,
 'strippedsize': 215,
 'time': 1231469665,
 'tx': [],
 'version': 1,
 'versionHex': '00000001',
 'weight': 860}

In [31]:
encoded_hash = ["0x4860eb18","0xbf1b1620","0xe37e9490","0xfc8a4275","0x14416fd7","0x5159ab86","0x688e9a83","0x00000000"]
previous_hash = ["0x7a9ee730","0xc339ff6f","0x80e7aa86","0x0b727c38","0x68a1cbc5","0x96a33dec","0x899fcc5b","0xcd35b4cd"]

print(''.join([(x[2:]) for x in previous_hash])[::-1])
(b"".join([binascii.unhexlify(x[2:]) for x in previous_hash])[::-1]).hex()

# assert(encoded_hash_inv == next_result['hash'])

# decoded_hash = ''.join([str(int(x, 16)) for x in encoded_hash])
# decoded_prev_hash = ''.join([str(int(x, 16)) for x in previous_hash])

# print(f"{decoded_hash}\n{decoded_prev_hash}")

# decoded_hash_inv = ''.join([str(int(x, 16)) for x in encoded_hash[::-1]])
# decoded_prev_hash_inv = ''.join([str(int(x, 16)) for x in previous_hash[::-1]])

# print(f"{decoded_hash_inv}\n{decoded_prev_hash_inv}")

dc4b53dcb5ccf998ced33a695cbc1a8683c727b068aa7e08f6ff933c037ee9a7


'cdb435cd5bcc9f89ec3da396c5cba168387c720b86aae7806fff39c330e79e7a'

In [14]:
next_result

{'bits': '1d00ffff',
 'chainwork': '0000000000000000000000000000000000000000000000000000000300030003',
 'confirmations': 652851,
 'difficulty': 1.0,
 'hash': '000000006a625f06636b8bb6ac7b960a8d03705d1ace08b1a19da3fdcc99ddbd',
 'height': 2,
 'mediantime': 1231469665,
 'merkleroot': '9b0fc92260312ce44e74ef369f5c66bbb85848f2eddd5a7a1cde251e54ccfdd5',
 'nTx': 1,
 'nextblockhash': '0000000082b5015589a3fdf2d4baff403e6f0be035a5d9742c1cae6295464449',
 'nonce': 1639830024,
 'previousblockhash': '00000000839a8e6886ab5951d76f411475428afc90947ee320161bbf18eb6048',
 'size': 215,
 'strippedsize': 215,
 'time': 1231469744,
 'tx': [],
 'version': 1,
 'versionHex': '00000001',
 'weight': 860}